# 2. About exploding gradients

### About this notebook

This notebook was used in the 50.039 Deep Learning course at the Singapore University of Technology and Design.

**Author:** Matthieu DE MARI (matthieu_demari@sutd.edu.sg)

**Version:** 1.1 (27/01/2022)

**Requirements:**
- Python 3 (tested on v3.9.6)
- Matplotlib (tested on v3.5.1)
- Numpy (tested on v1.22.1)

### Imports

In [1]:
# Matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
# Numpy
import numpy as np
# Removing unecessary warnings (optional, just makes notebook outputs more readable)
import warnings
warnings.filterwarnings("ignore")

### Blast from the past - Mock dataset generation

As in previous notebook.

In [2]:
# All helper functions
min_surf = 40
max_surf = 200
def surface(min_surf, max_surf):
    return round(np.random.uniform(min_surf, max_surf), 2)
min_dist = 50
max_dist = 1000
def distance(min_dist, max_dist):
    return round(np.random.uniform(min_dist, max_dist), 2)
def price(surface, distance):
    return round((100000 + 14373*surface + (1000 - distance)*1286)*(1 + np.random.uniform(-0.1, 0.1)))/1000000
n_points = 100
def create_dataset(n_points, min_surf, max_surf, min_dist, max_dist):
    surfaces_list = np.array([surface(min_surf, max_surf) for _ in range(n_points)])
    distances_list = np.array([distance(min_dist, max_dist) for _ in range(n_points)])
    inputs = np.array([[s, d] for s, d in zip(surfaces_list, distances_list)])
    outputs = np.array([price(s, d) for s, d in zip(surfaces_list, distances_list)]).reshape(n_points, 1)
    return surfaces_list, distances_list, inputs, outputs

In [3]:
# Generate dataset
np.random.seed(47)
surfaces_list, distances_list, inputs, outputs = create_dataset(n_points, min_surf, max_surf, min_dist, max_dist)
# Check a few entries of the dataset
print(surfaces_list.shape)
print(distances_list.shape)
print(inputs.shape)
print(outputs.shape)
print(inputs[0:10, :])
print(outputs[0:10])

(100,)
(100,)
(100, 2)
(100, 1)
[[ 58.16 572.97]
 [195.92 809.8 ]
 [156.6  349.04]
 [ 96.23  86.82]
 [153.22 817.92]
 [167.94 806.25]
 [143.29 315.92]
 [106.34 482.67]
 [152.96 427.77]
 [ 79.46 955.76]]
[[1.581913]
 [3.450274]
 [2.978769]
 [2.808258]
 [2.556398]
 [3.023983]
 [3.099523]
 [2.121069]
 [3.136544]
 [1.273443]]


### ShallowNeuralNet class with intializations

As in previous notebook.

In [4]:
class ShallowNeuralNet():
    
    def __init__(self, n_x, n_h, n_y, init_type = "Normal"):
        # Network dimensions
        self.n_x = n_x
        self.n_h = n_h
        self.n_y = n_y
        # Initialize parameters
        self.init_type = init_type
        self.init_parameters()
        # Loss, initialized as infinity before first calculation is made
        self.loss = float("Inf")
        
    def init_parameters(self):
        if(self.init_type == "Normal"):
            self.init_parameters_normal()
        elif(self.init_type == "Xavier"):
            self.init_parameters_xavier()
        elif(self.init_type == "He"):
            self.init_parameters_he()
        elif(self.init_type == "LeCun"):
            self.init_parameters_lecun()
        elif(self.init_type == "Weird"):
            self.init_parameters_weird()
        else:
            assert False, "Invalid initialization of parameters, check your init_type."
            
    def init_parameters_normal(self):
        # Weights and biases matrices (randomly initialized)
        self.W1 = np.random.randn(self.n_x, self.n_h)*0.1
        self.b1 = np.random.randn(1, self.n_h)*0.1
        self.W2 = np.random.randn(self.n_h, self.n_y)*0.1
        self.b2 = np.random.randn(1, self.n_y)*0.1
        
    def init_parameters_xavier(self):
        # Weights and biases matrices (Xavier initialized)
        var = np.sqrt(2.0/(self.n_x + self.n_y))
        self.W1 = np.random.randn(self.n_x, self.n_h)*var
        self.b1 = np.random.randn(1, self.n_h)*var
        self.W2 = np.random.randn(self.n_h, self.n_y)*var
        self.b2 = np.random.randn(1, self.n_y)*var
        
    def init_parameters_he(self):
        # Weights and biases matrices (He initialized)
        range1 = np.sqrt(4/(self.n_x + self.n_h))
        self.W1 = np.random.randn(self.n_x, self.n_h)*range1
        self.b1 = np.random.randn(1, self.n_h)*range1
        range2 = np.sqrt(4/(self.n_h + self.n_y))
        self.W2 = np.random.randn(self.n_h, self.n_y)*range2
        self.b2 = np.random.randn(1, self.n_y)*range2
        
    def init_parameters_lecun(self):
        # Weights and biases matrices (LeCun initialized)
        range1 = np.sqrt(1/self.n_x)
        self.W1 = np.random.randn(self.n_x, self.n_h)*range1
        self.b1 = np.zeros((1, self.n_h))
        range2 = np.sqrt(1/self.n_h)
        self.W2 = np.random.randn(self.n_h, self.n_y)*range2
        self.b2 = np.zeros((1, self.n_y))
        
    def init_parameters_const(self, const_val = 0.1):
        # Weights and biases matrices (Constant initialized)
        self.W1 = np.ones(shape = (n_x, n_h))*const_val
        self.b1 = np.ones(shape = (1, n_h))*const_val
        self.W2 = np.ones(shape = (n_h, n_y))*const_val
        self.b2 = np.ones(shape = (1, n_y))*const_val
        
    def init_parameters_weird(self):
        # Weights and biases matrices (Weird? initialized)
        init_val = np.sqrt(6.0/(self.n_x + self.n_y))
        self.W1 = np.random.uniform(-init_val, init_val, (self.n_x, self.n_h))
        self.b1 = np.random.uniform(-init_val, init_val, (1, self.n_h))
        self.W2 = np.random.uniform(-init_val, init_val, (self.n_h, self.n_y))
        self.b2 = np.random.uniform(-init_val, init_val, (1, self.n_y))
        
    def forward(self, inputs):
        # Wx + b operation for the first layer
        Z1 = np.matmul(inputs, self.W1)
        Z1_b = Z1 + self.b1
        # Wx + b operation for the second layer
        Z2 = np.matmul(Z1_b, self.W2)
        Z2_b = Z2 + self.b2
        return Z2_b
    
    def MSE_loss(self, inputs, outputs):
        # MSE loss function as before
        outputs_re = outputs.reshape(-1, 1)
        pred = self.forward(inputs)
        losses = (pred - outputs_re)**2
        self.loss = np.sum(losses)/outputs.shape[0]
        return self.loss
    
    def backward(self, inputs, outputs, alpha = 1e-5):
        # Get the number of samples in dataset
        m = inputs.shape[0]
        
        # Forward propagate
        Z1 = np.matmul(inputs, self.W1)
        Z1_b = Z1 + self.b1
        Z2 = np.matmul(Z1_b, self.W2)
        y_pred = Z2 + self.b2
    
        # Compute error term
        epsilon = y_pred - outputs
    
        # Compute the gradient for W2 and b2
        dL_dW2 = (2/m)*np.matmul(Z1_b.T, epsilon)
        dL_db2 = (2/m)*np.sum(epsilon, axis = 0, keepdims = True)

        # Compute the loss derivative with respect to the first layer
        dL_dZ1 = np.matmul(epsilon, self.W2.T)

        # Compute the gradient for W1 and b1
        dL_dW1 = (2/m)*np.matmul(inputs.T, dL_dZ1)
        dL_db1 = (2/m)*np.sum(dL_dZ1, axis = 0, keepdims = True)
        
        # Display (for this notebook only)
        print("- Gradients: ")
        print(alpha*dL_dW1)
        print(alpha*dL_db1)
        print(alpha*dL_dW2)
        print(alpha*dL_db2)
        print("- Parameters: ")
        print(self.W1)
        print(self.b1)
        print(self.W2)
        print(self.b2)
        
        # Update the weights and biases using gradient descent
        self.W1 -= alpha*dL_dW1
        self.b1 -= alpha*dL_db1
        self.W2 -= alpha*dL_dW2
        self.b2 -= alpha*dL_db2
        
        # Update loss
        self.MSE_loss(inputs, outputs)
    
    def train(self, inputs, outputs, N_max = 1000, alpha = 1e-5, delta = 1e-5, display = True):
        # List of losses, starts with the current loss
        self.losses_list = [self.loss]
        # Repeat iterations
        for iteration_number in range(1, N_max + 1):
            # Backpropagate
            self.backward(inputs, outputs, alpha)
            new_loss = self.loss
            # Update losses list
            self.losses_list.append(new_loss)
            # Display
            if(display and iteration_number % (N_max*0.05) == 1):
                print("Iteration {} - Loss = {}".format(iteration_number, new_loss))
            # Check for delta value and early stop criterion
            difference = abs(self.losses_list[-1] - self.losses_list[-2])
            if(difference < delta):
                if(display):
                    print("Stopping early - loss evolution was less than beta on iteration {}.".format(iteration_number))
                break
        else:
            # Else on for loop will execute if break did not trigger
            if(display):
                print("Stopping - Maximal number of iterations reached.")
    
    def show_losses_over_training(self):
        # Initialize matplotlib
        fig, axs = plt.subplots(1, 2, figsize = (15, 5))
        axs[0].plot(list(range(len(self.losses_list))), self.losses_list)
        axs[0].set_xlabel("Iteration number")
        axs[0].set_ylabel("Loss")
        axs[1].plot(list(range(len(self.losses_list))), self.losses_list)
        axs[1].set_xlabel("Iteration number")
        axs[1].set_ylabel("Loss (in logarithmic scale)")
        axs[1].set_yscale("log")
        # Display
        plt.show()

### The signs of an exploding gradient

In the previous notebook, we have tried different initialization methods and checked their final loss values after training.

In the model class above, we have added another initializer, called weird, whose initialization relies on a uniform distribution.

```
def init_parameters_weird(self):
        # Weights and biases matrices (Weird? initialized)
        init_val = np.sqrt(6.0/(self.n_x + self.n_y))
        self.W1 = np.random.uniform(-init_val, init_val, (self.n_x, self.n_h))
        self.b1 = np.random.uniform(-init_val, init_val, (1, self.n_h))
        self.W2 = np.random.uniform(-init_val, init_val, (self.n_h, self.n_y))
        self.b2 = np.random.uniform(-init_val, init_val, (1, self.n_y))
```

After training, we observe some NaN values in the loss function. These NaN value are a typical symptom for a phenomenon called the **exploding gradient syndrom**. This typically occurs when the gradient descent rule has changes (in alpha\*dL_dW1, alpha\*dL_db1, alpha\*dL_dW2, and alpha\*dL_db2) that are far greater than the values in the matrices $ W_1 $, $ b_1 $, $ W_2 $ and $ b_2 $.

In order to observe that, we have added the following display to the **backward()** method of our model.

```
# Display (for this notebook only)
print("- Gradients: ")
print(alpha*dL_dW1)
print(alpha*dL_db1)
print(alpha*dL_dW2)
print(alpha*dL_db2)
print("- Parameters: ")
print(self.W1)
print(self.b1)
print(self.W2)
print(self.b2)
```

When running this Weird initialization model, we can observe this eventually happens in the first few iterations of training, eventually leading to values so large in our matrices $ W_1 $, $ b_1 $, $ W_2 $ and $ b_2 $ that they end up being replaced by NaNs. This in turn gives a NaN value for the loss function.

In [5]:
# Define and train neural network structure (Weird initialization)
n_x = 2
n_h = 4
n_y = 1
init_type = "Weird"
np.random.seed(37)
shallow_neural_net_weird = ShallowNeuralNet(n_x, n_h, n_y, init_type)
# Train and show final loss
shallow_neural_net_weird.train(inputs, outputs, N_max = 500, alpha = 1e-6, delta = 1e-6, display = True)

- Gradients: 
[[ 0.0347825  -0.28526197 -0.24439365  0.18661634]
 [ 0.17610429 -1.44428536 -1.23736846  0.94484113]]
[[ 0.00026903 -0.00220639 -0.00189029  0.0014434 ]]
[[-0.64676422]
 [-0.55320161]
 [ 1.42348809]
 [-0.81598256]]
[[-0.00168314]]
- Parameters: 
[[ 1.25722625 -0.1015457  -0.86890687  0.23163369]
 [ 0.33964944  0.52106421 -1.12164798  0.69431032]]
[[-0.61665631  0.71679298  0.82789654  0.3603433 ]]
[[-0.15983769]
 [ 1.31087797]
 [ 1.12307381]
 [-0.85756698]]
[[0.27139079]]
Iteration 1 - Loss = 5638943.191688167
- Gradients: 
[[ 0.2638808   1.01020334 -0.16280394 -0.0225359 ]
 [ 1.39767888  5.35067306 -0.8623122  -0.11936434]]
[[ 0.00207055  0.00792661 -0.00127745 -0.00017683]]
[[ 1.12929903e+00]
 [ 5.74397848e+00]
 [-2.74959803e-03]
 [-6.93203740e-01]]
[[0.00425229]]
- Parameters: 
[[ 1.22244375  0.18371627 -0.62451323  0.04501735]
 [ 0.16354515  1.96534957  0.11572048 -0.25053081]]
[[-0.61692534  0.71899936  0.82978683  0.3588999 ]]
[[ 0.48692653]
 [ 1.86407958]
 [-0.300

[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
Iteration 251 - Loss = nan
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[

In [6]:
# Showing final loss
print(shallow_neural_net_weird.loss)

nan


### Unlucky initializations

In fact, that was just unlucky initialization, using a different seed for the random (e.g. 17 instead of the previous 37) already resolves the problem!

Yes, sometimes, it is just that simple!

In [7]:
# Define and train neural network structure (Weird initialization)
n_x = 2
n_h = 4
n_y = 1
init_type = "Weird"
np.random.seed(17)
shallow_neural_net_weird2 = ShallowNeuralNet(n_x, n_h, n_y, init_type)
# Train and show final loss
shallow_neural_net_weird2.train(inputs, outputs, N_max = 500, alpha = 1e-6, delta = 1e-6, display = True)

- Gradients: 
[[ 0.12749416  0.132134   -0.15718013  0.05337977]
 [ 0.68599444  0.71095953 -0.84572264  0.28721492]]
[[ 0.00100453  0.00104109 -0.00123843  0.00042058]]
[[ 0.46760632]
 [ 0.30491179]
 [ 0.15233671]
 [-0.01007841]]
[[0.00097559]]
- Parameters: 
[[-0.58077508  0.08651241 -0.87251097 -1.22216235]
 [ 0.81171746  0.44217797  0.38896783  0.21383728]]
[[-1.30372695 -0.40216386  1.26058241 -1.24438156]]
[[ 1.02966656]
 [ 1.06713876]
 [-1.26941601]
 [ 0.43110495]]
[[0.14637497]]
Iteration 1 - Loss = 1356587.2512042476
- Gradients: 
[[-0.14688568 -0.19919613  0.37155289 -0.11529639]
 [-0.79169523 -1.07364197  2.00262316 -0.62143297]]
[[-0.00115959 -0.00157255  0.00293322 -0.00091021]]
[[ 0.01069856]
 [ 0.39134913]
 [-1.55479758]
 [ 0.43926795]]
[[-0.0020631]]
- Parameters: 
[[-0.70826923 -0.04562159 -0.71533084 -1.27554212]
 [ 0.12572302 -0.26878155  1.23469047 -0.07337764]]
[[-1.30473148 -0.40320495  1.26182085 -1.24480214]]
[[ 0.56206024]
 [ 0.76222697]
 [-1.42175272]
 [ 0.4411

[[ 1.15665769e-03]
 [-9.11508886e-06]
 [ 1.44754278e-03]
 [ 2.04477257e-03]]
[[-8.52704533e-06]]
- Parameters: 
[[-0.58702444  0.12384311 -1.0390142  -1.17961935]
 [ 0.66251697  0.58145044 -0.61928668  0.48842586]]
[[-1.3038281  -0.40189612  1.25921473 -1.24402413]]
[[ 0.24187079]
 [ 0.14658759]
 [ 0.21822835]
 [-0.15506397]]
[[0.14754326]]
- Gradients: 
[[-3.73567496e-04 -2.27505400e-04 -3.36425046e-04  2.43819162e-04]
 [ 7.45094801e-05  4.53768308e-05  6.71012751e-05 -4.86306738e-05]]
[[-2.00098846e-06 -1.21861694e-06 -1.80203750e-06  1.30600048e-06]]
[[ 1.12630004e-03]
 [-9.25039437e-06]
 [ 1.40974069e-03]
 [ 1.99259188e-03]]
[[-8.31271705e-06]]
- Parameters: 
[[-0.58663917  0.12407661 -1.03866659 -1.17986634]
 [ 0.66244013  0.58140387 -0.61935601  0.48847513]]
[[-1.30382604 -0.40189487  1.25921659 -1.24402546]]
[[ 0.24071413]
 [ 0.14659671]
 [ 0.21678081]
 [-0.15710874]]
[[0.14755179]]
- Gradients: 
[[-3.62260155e-04 -2.21670260e-04 -3.25644066e-04  2.40563438e-04]
 [ 7.22560039e-0

[[-0.57342233  0.13292925 -1.02722048 -1.19164527]
 [ 0.65981192  0.57964402 -0.6216324   0.49081533]]
[[-1.30374889 -0.40184283  1.2592832  -1.24409565]]
[[ 0.19737482]
 [ 0.14724   ]
 [ 0.16241063]
 [-0.23486305]]
[[0.14790609]]
- Gradients: 
[[-1.63018601e-06 -1.21614480e-06 -1.34138396e-06  1.93996545e-06]
 [ 2.66565146e-07  1.98861856e-07  2.19340743e-07 -3.17219734e-07]]
[[-4.67760641e-08 -3.48956908e-08 -3.84892653e-08  5.56647815e-08]]
[[ 5.93634738e-06]
 [-2.19852309e-07]
 [ 7.34639316e-06]
 [ 1.08002666e-05]]
[[-2.36998351e-07]]
- Parameters: 
[[-0.57342066  0.1329305  -1.0272191  -1.19164726]
 [ 0.65981164  0.57964382 -0.62163263  0.49081565]]
[[-1.30374884 -0.40184279  1.25928324 -1.24409571]]
[[ 0.19736873]
 [ 0.14724023]
 [ 0.16240309]
 [-0.23487413]]
[[0.14790633]]
- Gradients: 
[[-1.58831012e-06 -1.18494212e-06 -1.30690694e-06  1.89027577e-06]
 [ 2.58195874e-07  1.92624327e-07  2.12450942e-07 -3.07283446e-07]]
[[-4.65558261e-08 -3.47324863e-08 -3.83074638e-08  5.5406906

In [8]:
# Showing final loss
print(shallow_neural_net_weird2.loss)

0.12422436714449628


 ### Changing the learning rate to create an exploding gradient problem
    
As we said before, the exploding gradient problem typically occurs when the gradient descent rule has changes (in alpha\*dL_dW1, alpha\*dL_db1, alpha\*dL_dW2, and alpha\*dL_db2) that are far greater than the values in the matrices $ W_1 $, $ b_1 $, $ W_2 $ and $ b_2 $.

Another very easy way to force the apparition of the exploding gradient consists of using a value of a learning rate $ \alpha $ that is simply far too big. Typically, we can force an exploding gradient problem in the random initialization by doing replacing $ \alpha = 1e^{-5} $ with $ \alpha = 1e^{-2} $, as shown below!

In [9]:
# Define and train neural network structure (random normal initialization)
n_x = 2
n_h = 4
n_y = 1
init_type = "Normal"
np.random.seed(37)
shallow_neural_net_normal1 = ShallowNeuralNet(n_x, n_h, n_y, init_type)
# Train and show final loss
shallow_neural_net_normal1.train(inputs, outputs, N_max = 1000, alpha = 1e-6, delta = 1e-6, display = True)
print(shallow_neural_net_normal1.loss)

- Gradients: 
[[ 1.42356523e-05 -1.73577846e-06 -9.53259136e-07  1.24839272e-06]
 [ 1.32743203e-04 -1.61856153e-05 -8.88885652e-06  1.16408890e-05]]
[[ 1.27309997e-07 -1.55231348e-08 -8.52503385e-09  1.11644251e-08]]
[[ 3.86686371e-04]
 [ 2.71147622e-04]
 [ 8.05690592e-05]
 [-1.49929229e-04]]
[[2.45019007e-06]]
- Parameters: 
[[-0.00544636  0.06743081  0.0346647  -0.13003462]
 [ 0.15185119  0.09898237  0.02776809 -0.04485894]]
[[ 0.09619662 -0.08275786  0.05346571  0.12283862]]
[[ 0.05195923]
 [-0.00633548]
 [-0.00347934]
 [ 0.00455655]]
[[0.14480251]]
Iteration 1 - Loss = 6.635227700991098
- Gradients: 
[[ 1.33509060e-05 -1.71029927e-06 -9.21574694e-07  1.21839677e-06]
 [ 1.27650738e-04 -1.63525206e-05 -8.81136384e-06  1.16493403e-05]]
[[ 1.20261279e-07 -1.54059042e-08 -8.30129070e-09  1.09749821e-08]]
[[ 3.74339400e-04]
 [ 2.62301797e-04]
 [ 7.78514894e-05]
 [-1.44438980e-04]]
[[2.33188559e-06]]
- Parameters: 
[[-0.0054606   0.06743254  0.03466566 -0.13003587]
 [ 0.15171844  0.098998

[[-1.175471e-06]]
- Parameters: 
[[-0.004855    0.06718966  0.03457211 -0.12991079]
 [ 0.1477254   0.09997331  0.02820287 -0.04546867]]
[[ 0.09619929 -0.08275914  0.05346523  0.12283925]]
[[ 0.03821603]
 [-0.01435874]
 [-0.00548384]
 [ 0.00664166]]
[[0.1448784]]
- Gradients: 
[[-7.16946139e-06  2.69444349e-06  1.02852898e-06 -1.24284325e-06]
 [ 3.88695744e-06 -1.46080530e-06 -5.57621855e-07  6.73813351e-07]]
[[-4.49265237e-08  1.68843896e-08  6.44514685e-09 -7.78812013e-09]]
[[ 1.58277388e-05]
 [-2.33974670e-06]
 [-3.68146743e-06]
 [ 1.96105479e-05]]
[[-1.17608324e-06]]
- Parameters: 
[[-0.00484783  0.06718697  0.03457108 -0.12990955]
 [ 0.14772149  0.09997477  0.02820343 -0.04546935]]
[[ 0.09619933 -0.08275916  0.05346523  0.12283925]]
[[ 0.03820012]
 [-0.01435646]
 [-0.00548018]
 [ 0.00662208]]
[[0.14487957]]
- Gradients: 
[[-7.16898464e-06  2.69494183e-06  1.02819571e-06 -1.23959392e-06]
 [ 3.86608754e-06 -1.45332729e-06 -5.54485022e-07  6.68487775e-07]]
[[-4.49303868e-08  1.6890087

[[-4.35862845e-08  1.65905314e-08  6.01859128e-09 -5.47299680e-09]]
[[ 1.32936781e-05]
 [-3.88929755e-06]
 [-4.11477804e-06]
 [ 2.03246151e-05]]
[[-1.18415776e-06]]
- Parameters: 
[[-0.00414866  0.06692223  0.0344725  -0.12980443]
 [ 0.14738601  0.10010178  0.02825076 -0.04552   ]]
[[ 0.09620373 -0.08276082  0.05346461  0.12283991]]
[[ 0.03680784]
 [-0.01401041]
 [-0.00508259]
 [ 0.00462185]]
[[0.14499691]]
- Gradients: 
[[-6.90718615e-06  2.62934704e-06  9.53347656e-07 -8.63812253e-07]
 [ 3.15165155e-06 -1.19973394e-06 -4.34999079e-07  3.94145338e-07]]
[[-4.35663338e-08  1.65843236e-08  6.01313781e-09 -5.44840288e-09]]
[[ 1.32877319e-05]
 [-3.89000242e-06]
 [-4.11454257e-06]
 [ 2.03226127e-05]]
[[-1.18404337e-06]]
- Parameters: 
[[-0.00414175  0.0669196   0.03447154 -0.12980356]
 [ 0.14738286  0.10010298  0.02825119 -0.04552039]]
[[ 0.09620377 -0.08276084  0.0534646   0.12283992]]
[[ 0.03679454]
 [-0.01400652]
 [-0.00507848]
 [ 0.00460152]]
[[0.14499809]]
- Gradients: 
[[-6.90379624e-

[[-1.17657241e-06]]
- Parameters: 
[[-0.00375349  0.06677147  0.03441864 -0.12976055]
 [ 0.14720618  0.10017039  0.02827527 -0.04553997]]
[[ 0.09620623 -0.08276177  0.05346427  0.12284019]]
[[ 0.03604445]
 [-0.01378528]
 [-0.00484482]
 [ 0.00344729]]
[[0.14506538]]
- Gradients: 
[[-6.70898627e-06  2.56607860e-06  9.01337617e-07 -6.38125194e-07]
 [ 3.05104012e-06 -1.16697344e-06 -4.09900560e-07  2.90199665e-07]]
[[-4.23885124e-08  1.62129195e-08  5.69480384e-09 -4.03178314e-09]]
[[ 1.30492227e-05]
 [-3.85264118e-06]
 [-4.07711514e-06]
 [ 2.01603446e-05]]
[[-1.1764328e-06]]
- Parameters: 
[[-0.00374678  0.0667689   0.03441773 -0.12975991]
 [ 0.14720312  0.10017156  0.02827568 -0.04554026]]
[[ 0.09620627 -0.08276179  0.05346426  0.1228402 ]]
[[ 0.03603139]
 [-0.01378142]
 [-0.00484074]
 [ 0.00342713]]
[[0.14506656]]
- Gradients: 
[[-6.70557348e-06  2.56498521e-06  9.00446458e-07 -6.34278385e-07]
 [ 3.04956693e-06 -1.16650635e-06 -4.09505875e-07  2.88457712e-07]]
[[-4.23681280e-08  1.62064

[[-0.00282771  0.06641524  0.03429848 -0.12970701]
 [ 0.1467836   0.100333    0.02833011 -0.04556438]]
[[ 0.09621209 -0.08276403  0.0534635   0.12284053]]
[[ 0.03420557]
 [-0.01325011]
 [-0.00427094]
 [ 0.00059709]]
[[0.1452322]]
- Gradients: 
[[-6.23356193e-06  2.41491046e-06  7.77897403e-07 -1.05261103e-07]
 [ 2.85868062e-06 -1.10746597e-06 -3.56739896e-07  4.82722205e-08]]
[[-3.95378967e-08  1.53171624e-08  4.93400523e-09 -6.67644386e-10]]
[[ 1.26746520e-05]
 [-3.62335352e-06]
 [-3.94593514e-06]
 [ 1.96948806e-05]]
[[-1.15631901e-06]]
- Parameters: 
[[-0.00282147  0.06641282  0.03429771 -0.12970691]
 [ 0.14678074  0.1003341   0.02833047 -0.04556443]]
[[ 0.09621213 -0.08276404  0.0534635   0.12284053]]
[[ 0.0341929 ]
 [-0.01324648]
 [-0.00426699]
 [ 0.00057739]]
[[0.14523335]]
- Gradients: 
[[-6.23033578e-06  2.41389520e-06  7.77063854e-07 -1.01655676e-07]
 [ 2.85738899e-06 -1.10707317e-06 -3.56381065e-07  4.66218546e-08]]
[[-3.95185006e-08  1.53111360e-08  4.92885126e-09 -6.44793479

[[-3.80289696e-08  1.48513979e-08  4.53384738e-09  1.10983338e-09]]
[[ 1.24856471e-05]
 [-3.49807326e-06]
 [-3.87589953e-06]
 [ 1.94497601e-05]]
[[-1.14548497e-06]]
- Parameters: 
[[-0.00233885  0.06622512  0.03423879 -0.12970957]
 [ 0.14655886  0.1004204   0.02835755 -0.0455632 ]]
[[ 0.09621519 -0.08276523  0.05346313  0.12284051]]
[[ 0.03319901]
 [-0.01296516]
 [-0.00395802]
 [-0.00096888]]
[[0.14532428]]
- Gradients: 
[[-5.97976651e-06  2.33551776e-06  7.12482883e-07  1.78083107e-07]
 [ 2.75651644e-06 -1.07661279e-06 -3.28436032e-07 -8.20916692e-08]]
[[-3.80101683e-08  1.48456337e-08  4.52887153e-09  1.13197879e-09]]
[[ 1.24832939e-05]
 [-3.49652263e-06]
 [-3.87503247e-06]
 [ 1.94467267e-05]]
[[-1.1453494e-06]]
- Parameters: 
[[-0.00233287  0.06622279  0.03423807 -0.12970975]
 [ 0.1465561   0.10042148  0.02835788 -0.04556312]]
[[ 0.09621523 -0.08276525  0.05346312  0.12284051]]
[[ 0.03318653]
 [-0.01296166]
 [-0.00395414]
 [-0.00098833]]
[[0.14532543]]
- Gradients: 
[[-5.97664828e-0

[[-5.80715088e-06  2.28207344e-06  6.68177542e-07  3.70458975e-07]
 [ 2.68632390e-06 -1.05566198e-06 -3.09091556e-07 -1.71370233e-07]]
[[-3.69689126e-08  1.45279114e-08  4.25368613e-09  2.35837948e-09]]
[[ 1.23529679e-05]
 [-3.41109241e-06]
 [-3.82724084e-06]
 [ 1.92795566e-05]]
[[-1.13781801e-06]]
- Parameters: 
[[-0.00200277  0.06609348  0.0341994  -0.12972503]
 [ 0.14640367  0.10048119  0.02837574 -0.04555606]]
[[ 0.09621733 -0.08276607  0.05346288  0.12284042]]
[[ 0.03249106]
 [-0.01276822]
 [-0.00373846]
 [-0.00207272]]
[[0.14538936]]
- Gradients: 
[[-5.80410514e-06  2.28113446e-06  6.67397150e-07  3.73850855e-07]
 [ 2.68508003e-06 -1.05529250e-06 -3.08749535e-07 -1.72949911e-07]]
[[-3.69505238e-08  1.45223271e-08  4.24883315e-09  2.38003699e-09]]
[[ 1.23506661e-05]
 [-3.40959175e-06]
 [-3.82640089e-06]
 [ 1.92766189e-05]]
[[-1.13768459e-06]]
- Parameters: 
[[-0.00199696  0.06609119  0.03419873 -0.1297254 ]
 [ 0.14640098  0.10048224  0.02837605 -0.04555589]]
[[ 0.09621736 -0.08276

[[ 1.19455252e-05]
 [-3.15034751e-06]
 [-3.68097621e-06]
 [ 1.87679640e-05]]
[[-1.11398468e-06]]
- Parameters: 
[[-0.00098954  0.06569098  0.03408975 -0.12984784]
 [ 0.14593246  0.10066838  0.02842673 -0.0454989 ]]
[[ 0.09622379 -0.08276864  0.05346218  0.12283963]]
[[ 0.03026796]
 [-0.01216814]
 [-0.00305161]
 [-0.00555355]]
[[0.14559539]]
- Gradients: 
[[-5.26738067e-06  2.11784737e-06  5.30624804e-07  9.70104898e-07]
 [ 2.46294169e-06 -9.90271051e-07 -2.48111544e-07 -4.53605302e-07]]
[[-3.37008835e-08  1.35500607e-08  3.39495583e-09  6.20676465e-09]]
[[ 1.19433710e-05]
 [-3.14899650e-06]
 [-3.68021630e-06]
 [ 1.87653051e-05]]
[[-1.1138575e-06]]
- Parameters: 
[[-0.00098427  0.06568886  0.03408922 -0.1298488 ]
 [ 0.14592999  0.10066937  0.02842697 -0.04549844]]
[[ 0.09622383 -0.08276865  0.05346217  0.12283963]]
[[ 0.03025601]
 [-0.01216499]
 [-0.00304793]
 [-0.00557231]]
[[0.14559651]]
- Gradients: 
[[-5.26455650e-06  2.11699961e-06  5.29909121e-07  9.73234111e-07]
 [ 2.46175723e-06

[[ 1.19047224e-05]
 [-3.12480988e-06]
 [-3.66660755e-06]
 [ 1.87176851e-05]]
[[-1.11157357e-06]]
- Parameters: 
[[-0.00088989  0.06565087  0.03407978 -0.12986674]
 [ 0.14588584  0.10068714  0.02843139 -0.04549005]]
[[ 0.09622443 -0.0827689   0.05346211  0.12283951]]
[[ 0.03004136]
 [-0.01210851]
 [-0.0029818 ]
 [-0.00590968]]
[[0.14561654]]
- Gradients: 
[[-5.21391556e-06  2.10181832e-06  5.17083281e-07  1.02932910e-06]
 [ 2.44049048e-06 -9.83803352e-07 -2.42032463e-07 -4.81800644e-07]]
[[-3.33761454e-08  1.34544937e-08  3.31003572e-09  6.58910513e-09]]
[[ 1.19025823e-05]
 [-3.12347345e-06]
 [-3.66585536e-06]
 [ 1.87150528e-05]]
[[-1.11144698e-06]]
- Parameters: 
[[-0.00088467  0.06564876  0.03407926 -0.12986777]
 [ 0.1458834   0.10068812  0.02843163 -0.04548957]]
[[ 0.09622446 -0.08276891  0.05346211  0.1228395 ]]
[[ 0.03002945]
 [-0.01210539]
 [-0.00297813]
 [-0.0059284 ]]
[[0.14561765]]
- Gradients: 
[[-5.21111293e-06  2.10097926e-06  5.16373857e-07  1.03243271e-06]
 [ 2.43931196e-0

[[-3.18191114e-08  1.30002747e-08  2.90389655e-09  8.42211958e-09]]
[[ 1.17066694e-05]
 [-3.00246004e-06]
 [-3.59762495e-06]
 [ 1.84761692e-05]]
[[-1.0998058e-06]]
- Parameters: 
[[-0.00041162  0.06545683  0.03403417 -0.1299766 ]
 [ 0.14566144  0.10077819  0.02845279 -0.0454385 ]]
[[ 0.0962275  -0.08277014  0.05346182  0.12283881]]
[[ 0.02893157]
 [-0.01182052]
 [-0.00264037]
 [-0.00765782]]
[[0.14572047]]
- Gradients: 
[[-4.95531097e-06  2.02488720e-06  4.51801453e-07  1.31530540e-06]
 [ 2.33106255e-06 -9.52541375e-07 -2.12535087e-07 -6.18742030e-07]]
[[-3.18026561e-08  1.29955096e-08  2.89961343e-09  8.44148947e-09]]
[[ 1.17045955e-05]
 [-3.00119335e-06]
 [-3.59690940e-06]
 [ 1.84736625e-05]]
[[-1.09968201e-06]]
- Parameters: 
[[-0.00040666  0.0654548   0.03403372 -0.12997791]
 [ 0.1456591   0.10077914  0.028453   -0.04543788]]
[[ 0.09622753 -0.08277015  0.05346182  0.1228388 ]]
[[ 0.02891987]
 [-0.01181752]
 [-0.00263677]
 [-0.0076763 ]]
[[0.14572157]]
- Gradients: 
[[-4.95261157e-0

In [10]:
# Show loss after training
print(shallow_neural_net_normal1.loss)

2.0907010654169245


In [11]:
# Define and train neural network structure (random normal initialization)
n_x = 2
n_h = 4
n_y = 1
init_type = "Normal"
np.random.seed(37)
shallow_neural_net_normal2 = ShallowNeuralNet(n_x, n_h, n_y, init_type)
# Train and show final loss
shallow_neural_net_normal2.train(inputs, outputs, N_max = 1000, alpha = 1e-2, delta = 1e-6, display = True)
print(shallow_neural_net_normal2.loss)

- Gradients: 
[[ 0.14235652 -0.01735778 -0.00953259  0.01248393]
 [ 1.32743203 -0.16185615 -0.08888857  0.11640889]]
[[ 1.27309997e-03 -1.55231348e-04 -8.52503385e-05  1.11644251e-04]]
[[ 3.86686371]
 [ 2.71147622]
 [ 0.80569059]
 [-1.49929229]]
[[0.0245019]]
- Parameters: 
[[-0.00544636  0.06743081  0.0346647  -0.13003462]
 [ 0.15185119  0.09898237  0.02776809 -0.04485894]]
[[ 0.09619662 -0.08275786  0.05346571  0.12283862]]
[[ 0.05195923]
 [-0.00633548]
 [-0.00347934]
 [ 0.00455655]]
[[0.14480251]]
Iteration 1 - Loss = 4354741.971946698
- Gradients: 
[[-17872.18478713 -12732.48994896  -3790.82479548   7045.27849364]
 [-96275.52328648 -68588.54399597 -20420.76249876  37952.15200341]]
[[-140.99869992 -100.45019962  -29.90688459   55.58218651]]
[[-30356.68076018]
 [  6976.86561045]
 [  3153.06278263]
 [ -4733.00388279]]
[[36.95995557]]
- Parameters: 
[[-0.14780288  0.08478859  0.04419729 -0.14251854]
 [-1.17558084  0.26083852  0.11665665 -0.16126783]]
[[ 0.09492352 -0.08260263  0.053550

[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan

[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan 

[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan 

[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan 

...

- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Para

[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan

- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Para

[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan 

[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]

In [12]:
# Show loss after training
print(shallow_neural_net_normal2.loss)

nan


### Controlling the exploding gradient problem

The lessons to be learned here are threefold:
- The exploding gradient problem is serious and can lead to a network being unable to train at all. A typical telltale sign is the apparition of NaNs in losses or network parameters.
- A typical way to prevent the exploding gradient problem consists of using a learning rate $ \alpha $ that is small enough to prevent changes in the gradient descent rule from being much larger than the parameters to be changed.
- And, more importantly, it is important to control the initiation of the network parameters. In fact, if we were to use the Xavier initialization from earlier, but reduce the value of the initial parameters in the matrices $ W_1 $, $ b_1 $, $ W_2 $ and $ b_2 $ before training, then we could make the exploding gradient problem disappear! We show it below, by first running the normal Weird initialization, and then running it again, but dividing all elements in $ W_1 $, $ b_1 $, $ W_2 $ and $ b_2 $ by - say - a factor 10. The second case has no exploding gradient issue and is able to train, although at a slower pace than other models!

In [13]:
# Define and train neural network structure (Weird initialization = some sort of a Uniform Xavier)
n_x = 2
n_h = 4
n_y = 1
init_type = "Weird"
np.random.seed(37)
shallow_neural_net_xavier1 = ShallowNeuralNet(n_x, n_h, n_y, init_type)
# Train and show final loss
shallow_neural_net_xavier1.train(inputs, outputs, N_max = 1000, alpha = 1e-6, delta = 1e-6, display = True)
print(shallow_neural_net_xavier1.loss)

- Gradients: 
[[ 0.0347825  -0.28526197 -0.24439365  0.18661634]
 [ 0.17610429 -1.44428536 -1.23736846  0.94484113]]
[[ 0.00026903 -0.00220639 -0.00189029  0.0014434 ]]
[[-0.64676422]
 [-0.55320161]
 [ 1.42348809]
 [-0.81598256]]
[[-0.00168314]]
- Parameters: 
[[ 1.25722625 -0.1015457  -0.86890687  0.23163369]
 [ 0.33964944  0.52106421 -1.12164798  0.69431032]]
[[-0.61665631  0.71679298  0.82789654  0.3603433 ]]
[[-0.15983769]
 [ 1.31087797]
 [ 1.12307381]
 [-0.85756698]]
[[0.27139079]]
Iteration 1 - Loss = 5638943.191688167
- Gradients: 
[[ 0.2638808   1.01020334 -0.16280394 -0.0225359 ]
 [ 1.39767888  5.35067306 -0.8623122  -0.11936434]]
[[ 0.00207055  0.00792661 -0.00127745 -0.00017683]]
[[ 1.12929903e+00]
 [ 5.74397848e+00]
 [-2.74959803e-03]
 [-6.93203740e-01]]
[[0.00425229]]
- Parameters: 
[[ 1.22244375  0.18371627 -0.62451323  0.04501735]
 [ 0.16354515  1.96534957  0.11572048 -0.25053081]]
[[-0.61692534  0.71899936  0.82978683  0.3588999 ]]
[[ 0.48692653]
 [ 1.86407958]
 [-0.300

[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan

[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
Iteration 151 - Loss = nan
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan

[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan 

[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan

...

[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan

[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
Iteration 651 - Loss = nan
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]


[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan

[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]

[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Gradients: 
[[nan nan nan nan]
 [nan nan nan nan]]
[[nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
[[nan]]
- Parameters: 
[[nan

In [14]:
# Show loss after training
print(shallow_neural_net_xavier1.loss)

nan


In [15]:
# Define and train neural network structure (Weird initialization = some sort of a Uniform Xavier)
n_x = 2
n_h = 4
n_y = 1
init_type = "Weird"
np.random.seed(37)
shallow_neural_net_xavier2 = ShallowNeuralNet(n_x, n_h, n_y, init_type)
# Divide initial values by 100!
shallow_neural_net_xavier2.W1 /= 100
shallow_neural_net_xavier2.b1 /= 100
shallow_neural_net_xavier2.W2 /= 100
shallow_neural_net_xavier2.b2 /= 100
# Train and show final loss
shallow_neural_net_xavier2.train(inputs, outputs, N_max = 1000, alpha = 1e-6, delta = 1e-6, display = True)
print(shallow_neural_net_xavier2.loss)

- Gradients: 
[[ 1.08643744e-06 -8.91020716e-06 -7.63367797e-06  5.82899367e-06]
 [ 4.17432348e-06 -3.42349093e-05 -2.93302128e-05  2.23962323e-05]]
[[ 8.17657760e-09 -6.70586248e-08 -5.74514080e-08  4.38692717e-08]]
[[-1.73842740e-05]
 [-1.29545695e-05]
 [ 3.51566967e-05]
 [-1.97254965e-05]]
[[-5.1155505e-06]]
- Parameters: 
[[ 0.01257226 -0.00101546 -0.00868907  0.00231634]
 [ 0.00339649  0.00521064 -0.01121648  0.0069431 ]]
[[-0.00616656  0.00716793  0.00827897  0.00360343]]
[[-0.00159838]
 [ 0.01310878]
 [ 0.01123074]
 [-0.00857567]]
[[0.00271391]]
Iteration 1 - Loss = 7.036477890046345
- Gradients: 
[[ 1.07422558e-06 -8.91572969e-06 -7.60698055e-06  5.81344543e-06]
 [ 4.12685197e-06 -3.42515552e-05 -2.92237341e-05  2.23335109e-05]]
[[ 8.08455398e-09 -6.70992195e-08 -5.72496560e-08  4.37516237e-08]]
[[-1.73650576e-05]
 [-1.30434025e-05]
 [ 3.50581006e-05]
 [-1.96533947e-05]]
[[-5.11359382e-06]]
- Parameters: 
[[ 0.01257118 -0.00100655 -0.00868144  0.00231051]
 [ 0.00339232  0.00524

[[0.00391503]]
- Gradients: 
[[-1.51062277e-06 -1.18520498e-05 -2.97996649e-06  3.38453701e-06]
 [-5.61704778e-06 -4.40702547e-05 -1.10806050e-05  1.25849461e-05]]
[[-1.13290322e-08 -8.88853636e-08 -2.23484890e-08  2.53825969e-08]]
[[-1.62925361e-05]
 [-3.49491896e-05]
 [ 1.97118216e-05]
 [-7.34368759e-06]]
[[-4.68387889e-06]]
- Parameters: 
[[ 0.01263565  0.00147842 -0.00744586  0.00122913]
 [ 0.00362991  0.0146372  -0.00650227  0.00282442]]
[[-0.00616609  0.00718667  0.00828831  0.00359526]]
[[ 0.00241873]
 [ 0.01897687]
 [ 0.00477136]
 [-0.00541914]]
[[0.00391972]]
- Gradients: 
[[-1.52018118e-06 -1.18690591e-05 -2.96645118e-06  3.37857894e-06]
 [-5.65163645e-06 -4.41260606e-05 -1.10284904e-05  1.25606737e-05]]
[[-1.14005100e-08 -8.90113153e-08 -2.22467273e-08  2.53374553e-08]]
[[-1.62984848e-05]
 [-3.50389640e-05]
 [ 1.96737002e-05]
 [-7.30827937e-06]]
[[-4.68189349e-06]]
- Parameters: 
[[ 0.01263716  0.00149027 -0.00744288  0.00122574]
 [ 0.00363553  0.01468127 -0.00649119  0.0028

[[-0.00616577  0.00718902  0.00828886  0.00359462]]
[[ 0.00284439]
 [ 0.01991462]
 [ 0.00427091]
 [-0.00523949]]
[[0.00404084]]
- Gradients: 
[[-1.76651411e-06 -1.23198500e-05 -2.62560330e-06  3.23128793e-06]
 [-6.53724619e-06 -4.55914230e-05 -9.71643251e-06  1.19578579e-05]]
[[-1.32413247e-08 -9.23463516e-08 -1.96808310e-08  2.42208835e-08]]
[[-1.64695666e-05]
 [-3.73585303e-05]
 [ 1.87279436e-05]
 [-6.41688660e-06]]
[[-4.62845021e-06]]
- Parameters: 
[[ 0.01267978  0.00180446 -0.00737006  0.00113977]
 [ 0.0037936   0.01584681 -0.00622103  0.00249291]]
[[-0.00616576  0.00718911  0.00828888  0.00359459]]
[[ 0.00286085]
 [ 0.01995189]
 [ 0.00425214]
 [-0.00523304]]
[[0.00404547]]
- Gradients: 
[[-1.77590168e-06 -1.23374852e-05 -2.61288862e-06  3.22590511e-06]
 [-6.57076136e-06 -4.56481751e-05 -9.66757775e-06  1.19357129e-05]]
[[-1.33114256e-08 -9.24766935e-08 -1.95851340e-08  2.41800524e-08]]
[[-1.64767220e-05]
 [-3.74470583e-05]
 [ 1.86932433e-05]
 [-6.38369869e-06]]
[[-4.62632056e-06]

[[-1.46328190e-08 -9.49762562e-08 -1.78072660e-08  2.34321638e-08]]
[[-1.66194342e-05]
 [-3.91162552e-05]
 [ 1.80559197e-05]
 [-5.76799388e-06]]
[[-4.58472304e-06]]
- Parameters: 
[[ 0.01271688  0.00205423 -0.00731993  0.00107614]
 [ 0.00393066  0.01676944 -0.00603585  0.00225788]]
[[-0.00616548  0.00719098  0.00828925  0.00359412]]
[[ 0.00319165]
 [ 0.02071581]
 [ 0.00388404]
 [-0.00511092]]
[[0.00413763]]
- Gradients: 
[[-1.96220914e-06 -1.26939283e-05 -2.36447831e-06  3.12236527e-06]
 [-7.23193193e-06 -4.67848322e-05 -8.71453802e-06  1.15078116e-05]]
[[-1.47017922e-08 -9.51088709e-08 -1.77157816e-08  2.33942266e-08]]
[[-1.66272678e-05]
 [-3.92033491e-05]
 [ 1.80234962e-05]
 [-5.73635876e-06]]
[[-4.58247189e-06]]
- Parameters: 
[[ 0.01271883  0.00206691 -0.00731755  0.00107302]
 [ 0.00393786  0.01681617 -0.00602709  0.00224635]]
[[-0.00616546  0.00719108  0.00828927  0.00359409]]
[[ 0.00320827]
 [ 0.02075493]
 [ 0.00386599]
 [-0.00510515]]
[[0.00414221]]
- Gradients: 
[[-1.97144957e-

[[-0.00616204  0.00720913  0.00829112  0.00359057]]
[[ 0.00615267]
 [ 0.02854939]
 [ 0.00119571]
 [-0.00452566]]
[[0.0048828]]
- Gradients: 
[[-3.38144070e-06 -1.56730656e-05 -6.47735304e-07  2.47926464e-06]
 [-1.18369723e-05 -5.48646745e-05 -2.26744324e-06  8.67884120e-06]]
[[-2.51986398e-08 -1.16796351e-07 -4.82695101e-09  1.84755854e-08]]
[[-1.79369039e-05]
 [-5.15487507e-05]
 [ 1.36747192e-05]
 [-1.37043246e-06]]
[[-4.08366187e-06]]
- Parameters: 
[[ 0.01318052  0.00449814 -0.00706904  0.0005989 ]
 [ 0.00559835  0.02557198 -0.00512687  0.00053568]]
[[-0.00616201  0.00720925  0.00829113  0.00359055]]
[[ 0.0061706 ]
 [ 0.02860089]
 [ 0.00118202]
 [-0.00452427]]
[[0.00488688]]
- Gradients: 
[[-3.38837818e-06 -1.56879251e-05 -6.39695716e-07  2.47640017e-06]
 [-1.18560975e-05 -5.48928010e-05 -2.23832594e-06  8.66504278e-06]]
[[-2.52492084e-08 -1.16901853e-07 -4.76682636e-09  1.84534136e-08]]
[[-1.79414710e-05]
 [-5.15971635e-05]
 [ 1.36540865e-05]
 [-1.35083197e-06]]
[[-4.07999708e-06]]

...

[[-0.00615823  0.00722575  0.00829131  0.00358824]]
[[ 0.00859789]
 [ 0.03582077]
 [-0.00048609]
 [-0.00448845]]
[[0.00540157]]
Iteration 601 - Loss = 3.8772659130496363
- Gradients: 
[[-4.08950650e-06 -1.70284068e-05  2.35974580e-07  2.13080488e-06]
 [-1.31874659e-05 -5.49116464e-05  7.60949209e-07  6.87122439e-06]]
[[-3.02281308e-08 -1.25867732e-07  1.74423747e-09  1.57501275e-08]]
[[-1.76973860e-05]
 [-5.39334843e-05]
 [ 1.10361556e-05]
 [ 6.50503585e-07]]
[[-3.50853572e-06]]
- Parameters: 
[[ 0.01369397  0.00673802 -0.0070454   0.00028596]
 [ 0.00732955  0.03312961 -0.0050443  -0.00052189]]
[[-0.0061582   0.00722588  0.00829131  0.00358822]]
[[ 0.0086156 ]
 [ 0.03587472]
 [-0.00049714]
 [-0.00448909]]
[[0.00540508]]
- Gradients: 
[[-4.09271837e-06 -1.70324147e-05  2.40907633e-07  2.12841541e-06]
 [-1.31871273e-05 -5.48800577e-05  7.76227272e-07  6.85795660e-06]]
[[-3.02495308e-08 -1.25887614e-07  1.78056299e-09  1.57312480e-08]]
[[-1.76879823e-05]
 [-5.39161002e-05]
 [ 1.10172095e-

[[0.00582619]]
- Gradients: 
[[-4.24650854e-06 -1.67250092e-05  7.04044838e-07  1.81691111e-06]
 [-1.18420255e-05 -4.66401949e-05  1.96333455e-06  5.06672895e-06]]
[[-3.09840369e-08 -1.22031617e-07  5.13696159e-09  1.32568298e-08]]
[[-1.54213570e-05]
 [-4.72105048e-05]
 [ 8.48874016e-06]
 [ 1.41967161e-06]]
[[-2.85805022e-06]]
- Parameters: 
[[ 1.42541029e-02  9.00093560e-03 -7.11173013e-03  2.33401483e-05]
 [ 9.02235667e-03  3.99729520e-02 -5.24260298e-03 -1.31741394e-03]]
[[-0.00615409  0.00724251  0.00829082  0.00358629]]
[[ 0.01084097]
 [ 0.04269751]
 [-0.00179737]
 [-0.00463842]]
[[0.00582905]]
- Gradients: 
[[-4.24578320e-06 -1.67168622e-05  7.06244494e-07  1.81457553e-06]
 [-1.18231461e-05 -4.65511062e-05  1.96666467e-06  5.05301155e-06]]
[[-3.09750687e-08 -1.21957701e-07  5.15239962e-09  1.32382175e-08]]
[[-1.53973713e-05]
 [-4.71306567e-05]
 [ 8.46951029e-06]
 [ 1.42159703e-06]]
[[-2.85316433e-06]]
- Parameters: 
[[ 1.42583494e-02  9.01766061e-03 -7.11243418e-03  2.15232372e-0

[[-2.49019851e-06]]
- Parameters: 
[[ 0.01458146  0.010276   -0.00717202 -0.00011144]
 [ 0.00988234  0.04332338 -0.00540066 -0.00167178]]
[[-0.00615171  0.00725176  0.00829038  0.00358532]]
[[ 0.01196663]
 [ 0.04611783]
 [-0.00240229]
 [-0.00475156]]
[[0.00603761]]
- Gradients: 
[[-4.12356580e-06 -1.58877868e-05  8.29291919e-07  1.63600597e-06]
 [-1.00833387e-05 -3.88503404e-05  2.02786416e-06  4.00051875e-06]]
[[-2.97787966e-08 -1.14735448e-07  5.98882535e-09  1.18146020e-08]]
[[-1.33314322e-05]
 [-4.00578381e-05]
 [ 6.99566408e-06]
 [ 1.44049759e-06]]
[[-2.4857119e-06]]
- Parameters: 
[[ 0.01458559  0.0102919  -0.00717285 -0.00011308]
 [ 0.00989245  0.04336233 -0.00540269 -0.00167579]]
[[-0.00615168  0.00725188  0.00829037  0.0035853 ]]
[[ 0.01197999]
 [ 0.04615798]
 [-0.0024093 ]
 [-0.00475301]]
[[0.0060401]]
- Gradients: 
[[-4.12129829e-06 -1.58751647e-05  8.30318528e-07  1.63378381e-06]
 [-1.00584126e-05 -3.87448192e-05  2.02646976e-06  3.98740167e-06]]
[[-2.97582104e-08 -1.146280

[[-2.24690078e-06]]
- Parameters: 
[[ 0.01481268  0.01116131 -0.00722056 -0.00020135]
 [ 0.01041774  0.04537366 -0.00551296 -0.00188005]]
[[-0.00615005  0.00725813  0.00829003  0.00358467]]
[[ 0.01268258]
 [ 0.04824698]
 [-0.00277337]
 [-0.00483112]]
[[0.00617262]]
- Gradients: 
[[-3.97594355e-06 -1.51220644e-05  8.70517649e-07  1.51355973e-06]
 [-8.61152810e-06 -3.27530008e-05  1.88546117e-06  3.27823118e-06]]
[[-2.84717625e-08 -1.08289220e-07  6.23378360e-09  1.08386129e-08]]
[[-1.16999040e-05]
 [-3.43195667e-05]
 [ 5.98435567e-06]
 [ 1.33311044e-06]]
[[-2.24287666e-06]]
- Parameters: 
[[ 0.01481666  0.01117645 -0.00722143 -0.00020286]
 [ 0.01042638  0.04540652 -0.00551485 -0.00188334]]
[[-0.00615002  0.00725824  0.00829003  0.00358466]]
[[ 0.01269431]
 [ 0.0482814 ]
 [-0.00277937]
 [-0.00483246]]
[[0.00617486]]
- Gradients: 
[[-3.97309405e-06 -1.51080435e-05  8.70964022e-07  1.51149912e-06]
 [-8.58517704e-06 -3.26458992e-05  1.88200436e-06  3.26609121e-06]]
[[-2.84469875e-08 -1.0817

[[ 0.01310825]
 [ 0.04948398]
 [-0.0029897 ]
 [-0.00488002]]
[[0.00625525]]
- Gradients: 
[[-3.86417851e-06 -1.45845412e-05  8.82199981e-07  1.43777435e-06]
 [-7.61206334e-06 -2.87301559e-05  1.73784987e-06  2.83227843e-06]]
[[-2.75076153e-08 -1.03821795e-07  6.28004572e-09  1.02349681e-08]]
[[-1.06241531e-05]
 [-3.04988779e-05]
 [ 5.35947012e-06]
 [ 1.22897622e-06]]
[[-2.09679235e-06]]
- Parameters: 
[[ 0.01496569  0.01174116 -0.00725476 -0.00025896]
 [ 0.0107351   0.04657635 -0.00558386 -0.00199956]]
[[-0.00614895  0.00726227  0.00828979  0.00358426]]
[[ 0.0131189 ]
 [ 0.04951458]
 [-0.00299507]
 [-0.00488125]]
[[0.00625735]]
- Gradients: 
[[-3.86116911e-06 -1.45703597e-05  8.82375755e-07  1.43585353e-06]
 [-7.58595890e-06 -2.86260836e-05  1.73358535e-06  2.82099166e-06]]
[[-2.74818352e-08 -1.03704400e-07  6.28030124e-09  1.02196742e-08]]
[[-1.05963288e-05]
 [-3.03997781e-05]
 [ 5.34364834e-06]
 [ 1.22601315e-06]]
[[-2.09313215e-06]]
- Parameters: 
[[ 0.01496956  0.01175574 -0.007255

In [16]:
# Show loss after training
print(shallow_neural_net_xavier2.loss)

2.2689392753336914


### What's next?

In the next notebook, we will investigate the vanishing gradient problem, which is the counterpart to the exploding gradient one.